In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [139]:
combinedDf = pd.DataFrame()
csvPaths = os.listdir('./dataset')

for path in csvPaths:
    df = pd.read_csv(f'./dataset/{path}')
    df.insert(0,'patientId', path.split('.')[0])
    patientType = path.split('.')[0].split('#')[0]
    df.insert(1, 'patient_type', 0 if patientType == 'adolescent'  else  1 if patientType == 'adult' else 2)

    combinedDf = pd.concat([combinedDf, df])

df = combinedDf.copy()

In [140]:
combinedDf.head()

,patientId,patient_type,Time,BG,CGM,CHO,insulin,LBGI,HBGI,Risk
0,adolescent#002,0,2025-01-01 00:00:00,152.41,152.523836,0.0,0.0153,0.0,3.215327,3.215327
1,adolescent#002,0,2025-01-01 00:03:00,152.41,149.193735,0.0,0.0153,0.0,3.215327,3.215327
2,adolescent#002,0,2025-01-01 00:06:00,152.41,148.157518,0.0,0.0153,0.0,3.215327,3.215327
3,adolescent#002,0,2025-01-01 00:09:00,152.41,148.174905,0.0,0.0153,0.0,3.215327,3.215327
4,adolescent#002,0,2025-01-01 00:12:00,152.41,148.957937,0.0,0.0153,0.0,3.215327,3.215327


In [141]:
df.isna().any(axis=1).sum()
#First reading is always null

np.int64(30)

In [142]:
df.duplicated(subset=['patientId', 'Time']).sum()
#no duplicates

np.int64(0)

In [143]:
df.groupby('patientId').apply(lambda g: g.interpolate().ffill())


/tmp/ipython-input-1023219416.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.groupby('patientId').apply(lambda g: g.interpolate().ffill())
/tmp/ipython-input-1023219416.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.groupby('patientId').apply(lambda g: g.interpolate().ffill())
/tmp/ipython-input-1023219416.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.groupby('patientId').apply(lambda g: g.interpolate().ffill())
/tmp/ipython-input-1023219416.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) befor

patientId  patient_type                 Time  \
patientId                                                                
adolescent#001 0     adolescent#001             0  2025-01-01 00:00:00   
               1     adolescent#001             0  2025-01-01 00:03:00   
               2     adolescent#001             0  2025-01-01 00:06:00   
               3     adolescent#001             0  2025-01-01 00:09:00   
               4     adolescent#001             0  2025-01-01 00:12:00   
...                             ...           ...                  ...   
child#010      2396       child#010             2  2025-01-05 23:48:00   
               2397       child#010             2  2025-01-05 23:51:00   
               2398       child#010             2  2025-01-05 23:54:00   
               2399       child#010             2  2025-01-05 23:57:00   
               2400       child#010             2  2025-01-06 00:00:00   

                             BG         CGM  CHO   insulin       LBGI  \
patientId                                                               
adolescent#001 0     149.020000  149.133836  0.0  0.013933   0.000000   
               1     149.020000  145.803735  0.0  0.013933   0.000000   
               2     149.020000  144.767518  0.0  0.013933   0.000000   
               3     149.020000  144.784905  0.0  0.013933   0.000000   
               4     149.020000  145.567937  0.0  0.013933   0.000000   
...                         ...         ...  ...       ...        ...   
child#010      2396   63.036484   68.540070  0.0  0.006625  11.534254   
               2397   63.175502   65.431243  0.0  0.006625  11.447199   
               2398   63.339229   59.334432  0.0  0.006625  11.345333   
               2399   63.526391   53.782989  0.0  0.006625  11.229757   
               2400   63.735772   53.780947  0.0  0.006625  11.101553   

                         HBGI       Risk  
patientId                                 
adolescent#001 0     2.755276   2.755276  
               1     2.755276   2.755276  
               2     2.755276   2.755276  
               3     2.755276   2.755276  
               4     2.755276   2.755276  
...                       ...        ...  
child#010      2396  0.000000  11.534254  
               2397  0.000000  11.447199  
               2398  0.000000  11.345333  
               2399  0.000000  11.229757  
               2400  0.000000  11.101553  

[72030 rows x 10 columns]

In [144]:
df['Time'] = pd.to_datetime(df['Time'])
df = df.sort_values(['patientId', 'Time'])
hour = df['Time'].dt.hour
df.insert(3, 'hour', hour)


In [147]:
df.insert(7,'lastMealTime',np.where(df['CHO'] > 0.0, df['Time'].dt.hour, pd.NaT))

In [148]:
df['lastMealTime'] = df.groupby('patientId')['lastMealTime'].ffill()
df['lastMealTime'] = pd.to_datetime(df['lastMealTime'])
df.insert(8,'hours_since_last_meal', (
    (df['Time'] - df['lastMealTime']).dt.total_seconds() / 3600
))
df['hours_since_last_meal'] = df['hours_since_last_meal'].fillna(-1)

In [154]:
df.drop(columns=['lastMealTime'], inplace=True)

In [156]:
df['CGM'].max()

473.6247467285575

In [157]:
df['CGM'] = df['CGM'].clip(lower=40, upper=400)

In [159]:
df['CGM'] = (
    df.groupby('patientId')['CGM']
      .transform(lambda x: (x - x.mean()) / x.std())
)


In [161]:
from sklearn.model_selection import train_test_split

patients= df['patientId'].unique()

train_patients, test_patients = train_test_split(patients, test_size=0.3, random_state=42)


In [163]:
train_df = df[df['patientId'].isin(train_patients)].copy()
test_df  = df[df['patientId'].isin(test_patients)].copy()


In [165]:
train_df.to_csv('./dataset/train.csv', index=False)
test_df.to_csv('./dataset/test.csv', index=False)